# Part 3: Machine Learning

THIS PART IS NOT FINISHED

In this part, I will do some more data preprocessing. Then, I will do a first test with a Ridge Linear Regression model. I highlight why a regular performance metric, such as Mean Squared Error does not work on this data set. I therefore, have to create a new performance metric. This one is based on how well the player did in the MVP-voting - which rank the player got. Based on this new value, I am able to determine the performance of the regression model.

## Data preprocessing

There is still some data cleaning to be done. Some of the columns contain missing data. This could cause problems when doing the machine learning, such as biased model estimates, inaccurate predictions, and reduced model performance.

In [2]:
import pandas as pd

In [3]:
stats = pd.read_csv("player_mvp_stats.csv", index_col=0)

I will be using the `isnull()`-method to determine how many rows each column inside the dataframe that are missing.

In [4]:
pd.isnull(stats)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15232,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15233,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
15234,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


To get a better overview, I will combine the `isnull()`-method with `sum()`-method.

In [5]:
pd.isnull(stats).sum()

Player        0
Pos           0
Age           0
Tm            0
G             0
GS            0
MP            0
FG            0
FGA           0
FG%          61
3P            0
3PA           0
3P%        2102
2P            0
2PA           0
2P%         105
eFG%         61
FT            0
FTA           0
FT%         545
ORB           0
DRB           0
TRB           0
AST           0
STL           0
BLK           0
TOV           0
PF            0
PTS           0
Year          0
Pts Won       0
Pts Max       0
Share         0
Team          0
W             0
L             0
W/L%          0
GB            0
PS/G          0
PA/G          0
SRS           0
dtype: int64

From the output above it looks like all the columns that contain missing data are columns containing percentages. An explanation for this could be that for the cases where there is missing data, the player has not attempted a shot. The reason for this is that each of the columns containing percentages are computed by dividing the shots made with the shots attempted. If the player has not attempted a shot that season this value will be assigned 'False' (dividing something on zero does not work).

I'm checking if this explanation holds true by looking at the "3P%" column. For the players having "3P%" equal to 'False' we would expect them to have zero attempts. 

To display this, I´m using boolean indexing to filter the rows in the "stats" dataframe where the "3P%" column has missing values (NaN). `pd.isnull(stats["3P%"])` returns a boolean series indicating which rows have missing values in the "3P%" column, and this series is used to filter the rows in the "stats" dataframe.

In [6]:
stats[pd.isnull(stats["3P%"])][["Player", "3PA"]].head()

,Player,3PA
2,Elden Campbell,0.0
3,Irving Thomas,0.0
18,Jack Haley,0.0
20,Keith Owens,0.0
30,Benoit Benjamin,0.0


This can also be done for the freethrow percentage (FT%).

In [7]:
stats[pd.isnull(stats["FT%"])][["Player", "FTA"]].head()

,Player,FTA
77,John Coker,0.0
92,Jason Sasser,0.0
103,Adrian Caldwell,0.0
119,Bruno Šundov,0.0
158,Jamal Robinson,0.0


This seems to hold true.

A quick and easy way to fix this is by replacing these values with zeroes. It is important to note that replacing missing values with zero may not always be appropriate in every situation. The decision to use zero or a different replacement value depends on the specific context and the analysis being performed.

It is not entirely correct to replace the NaN-values with zeroes for this caes, because no attempts does not mean a zero percent season average. It will, however, be a reasonable approximation.

In [8]:
stats = stats.fillna(0)

## Choosing predictors

In the next steps, I will decide which of the columns I will use as predictors for my machine learning model.

In [9]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

The following columns will be removed:
- Player names ('Player'), positions ('Pos') and team names ('Tm') because these contain strings.
- 'Share' because it contains the values that I want the algorithm to predict.
- 'Pts Won' and 'Pts Max' because I do not want to any variables that directly correlate with 'Share'.

In [10]:
predictors = ["Age", "G", "GS", "MP", "FG", "FGA", 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'W', 'L', 'W/L%',
       'GB', 'PS/G', 'PA/G', 'SRS']

Now, I will create the training and test sets. The training set is the dataset which the algorithm is going to train on. The test set will be used to assess the performance of the trained model.

Because I'm working with temporal data it is important to make sure that the test set is placed after the training set in the timeframe.

In [11]:
train = stats[~(stats["Year"] == 2023)]
test = stats[stats["Year"] == 2023]

I will do a first test with the Ridge linear regression model. This is imported from the scikit-learn library. 

The alpha parameter controls the regularization strength: higher values mean stronger regularization. The purpose of regularization is to prevent overfitting of the model to the training data. 

In [12]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

Training the model.

In [13]:
reg.fit(train[predictors],train["Share"])

Ridge(alpha=0.1)

And testing.

I'm converting the predictions array to a dataframe for readability.

In [14]:
predictions = reg.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)

In [15]:
predictions

,predictions
211,0.003186
212,0.031702
213,0.042326
214,0.225694
215,0.000023
...,...
15111,-0.012347
15112,0.004315
15113,-0.014852
15114,0.019197


By combining the test data with the predictions one can see how the model performed for the 2023 data. Joel Embiid won the MVP award but the algorithm did not predict him to win it.

In [16]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [17]:
combination.sort_values("Share", ascending=False).head(20)

,Player,Share,predictions
14849,Joel Embiid,0.915,0.202730
736,Nikola Jokić,0.674,0.173179
214,Giannis Antetokounmpo,0.606,0.225694
2898,Jayson Tatum,0.280,0.136303
1322,Shai Gilgeous-Alexander,0.046,0.150951
13682,Donovan Mitchell,0.030,0.087768
4274,Domantas Sabonis,0.027,0.092822
306,Luka Dončić,0.010,0.194360
6686,Stephen Curry,0.005,0.106164
10869,Jimmy Butler,0.003,0.112078


To assess the performance of the model I could use the default error metric from a scikit-learn called mean squared error. 

In [18]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combination["Share"], combination["predictions"])

0.0026601418847589735

Based on this number, it seems like the model predicts very well (very low error rate). The problem, however, is that the "Share" column contains a lot of zero values because most NBA players do not get any MVP votes. This means that by predicting zero for every player, the model acieves a low error rate. But I´m not interested in ranking the players having zero MVP votes. I want the model to be able to predict who is the one winner. Thus, this performance does not make any sense for this case.

In [19]:
combination["Share"].value_counts()

0.000    526
0.001      2
0.606      1
0.010      1
0.674      1
0.046      1
0.280      1
0.002      1
0.027      1
0.005      1
0.003      1
0.030      1
0.915      1
Name: Share, dtype: int64

I therefore have to create a different performance metric. I will do this by creating a new column in the dataset that assigns a rank for each player based on the MVP-share they got that year.

First, I will sort the dataset based on the values in the "Share" column in descending order (highest to lowest) and assign the sorted result to the variable "actual".

In [20]:
actual = combination.sort_values("Share", ascending = False)

And then assign a rank based on the position in the sorted dataset. The following code creates a list of numbers using the `range` function. The `range` function generates a sequence of numbers from 1 to the number of rows in the "actual" dataframe, which is given by `actual.shape[0]`. The '+ 1' is added to include the upper bound of the range.

In [21]:
actual["Rk"] = list(range(1, actual.shape[0] + 1))

In [22]:
actual.head()

,Player,Share,predictions,Rk
14849,Joel Embiid,0.915,0.202730,1
736,Nikola Jokić,0.674,0.173179,2
214,Giannis Antetokounmpo,0.606,0.225694,3
2898,Jayson Tatum,0.280,0.136303,4
1322,Shai Gilgeous-Alexander,0.046,0.150951,5


I will do the same for the "predicted" dataset, sorting on the values in the "predictions" column.

In [23]:
predicted = combination.sort_values("predictions", ascending = False)
predicted["Predicted_Rk"] = list(range(1, predicted.shape[0] + 1))

In [24]:
predicted.head(10)

,Player,Share,predictions,Predicted_Rk
214,Giannis Antetokounmpo,0.606,0.225694,1
14849,Joel Embiid,0.915,0.202730,2
306,Luka Dončić,0.010,0.194360,3
736,Nikola Jokić,0.674,0.173179,4
1322,Shai Gilgeous-Alexander,0.046,0.150951,5
11528,Kevin Durant,0.000,0.140956,6
8563,Anthony Davis,0.000,0.140512,7
8083,Damian Lillard,0.000,0.136720,8
2898,Jayson Tatum,0.280,0.136303,9
8570,LeBron James,0.000,0.132677,10


Then, I perform a merge operation between the "actual" and "predicted" dataframes using the "Player" column as the key.

In [25]:
actual.merge(predicted, on = "Player").head(10)

,Player,Share_x,predictions_x,Rk,Share_y,predictions_y,Predicted_Rk
0,Joel Embiid,0.915,0.202730,1,0.915,0.202730,2
1,Nikola Jokić,0.674,0.173179,2,0.674,0.173179,4
2,Giannis Antetokounmpo,0.606,0.225694,3,0.606,0.225694,1
3,Jayson Tatum,0.280,0.136303,4,0.280,0.136303,9
4,Shai Gilgeous-Alexander,0.046,0.150951,5,0.046,0.150951,5
5,Donovan Mitchell,0.030,0.087768,6,0.030,0.087768,22
6,Domantas Sabonis,0.027,0.092822,7,0.027,0.092822,18
7,Luka Dončić,0.010,0.194360,8,0.010,0.194360,3
8,Stephen Curry,0.005,0.106164,9,0.005,0.106164,14
9,Jimmy Butler,0.003,0.112078,10,0.003,0.112078,13


Because I want the model to be able to predict the MVP winner, I have to create a performance metric that takes these ranks into account. A way to do this is to use average precision. Here, I want the model to predict the top 5 players as correctly as possible.

TO BE CONTINUED ...